In [ ]:
import rioxarray
import rasterio as ri
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.style import use as usestyle
from glob import glob
import xesmf
from matplotlib.colors import BoundaryNorm
from matplotlib import colors

In [ ]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        # Note also that we must extrapolate beyond vmin/vmax
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1.]
        return np.ma.masked_array(np.interp(value, x, y,
                                            left=-np.inf, right=np.inf))

    def inverse(self, value):
        y, x = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y, left=-np.inf, right=np.inf)

In [ ]:
from matplotlib import font_manager as fm
fontdir = "/home/ascheb/libfonts/*.ttf"
for fpath in glob(fontdir):
    print(fpath)
    fm.fontManager.addfont(fpath)

usestyle("paperplots.mplstyle")
from matplotlib import rcParams
rcParams["figure.titlesize"] = 40
rcParams["axes.titlesize"] = 35
rcParams["axes.labelsize"] = 30
rcParams["xtick.labelsize"] = 27
rcParams["ytick.labelsize"] = 27

In [ ]:
def interp_nlcd(nlcdras, **gridprops):
    srfda = xr.DataArray(np.ones((ny,nx)), dims = ["y", "x"], 
                         coords = {"y": np.arange(-np.floor(ny/2)*dx, np.floor(ny/2)*dx, dx), "x": np.arange(-np.floor(nx/2)*dx, np.floor(nx/2)*dx, dx)})

    crsstring = f"+proj=stere +lat_0={theta0} +lon_0={phi0} +ellps=sphere +R=6367000"; print(crsstring)
    srfda.rio.set_spatial_dims(x_dim = "x", y_dim = "y", inplace = True)
    srfda.rio.write_crs(crsstring, inplace = True)
    srfda.rio.write_coordinate_system(inplace = True)
    srftran = srfda.rio.transform(recalc = True)
    srfda.rio.write_transform(srftran, inplace = True)
    vrtopts = {
    "resampling": ri.enums.Resampling.bilinear,
    "crs": ri.crs.CRS.from_user_input(crsstring),
    "transform": srftran,
    "height": ny,
    "width": nx
    }
    testvrt = WarpedVRT(nlcdras, **vrtopts)
    vrtds = rioxarray.open_rasterio(testvrt)
    return vrtds

In [ ]:
# raster = ri.open("/moonbow/ascheb/les/SNODAS/zz_ssmv01025SlL01T0024TTNATS2010010105DP001.dat")
# rastest2 = xr.open_dataset("/moonbow/ascheb/les/SNODAS/zz_ssmv01025SlL01T0024TTNATS2010010105DP001.dat", engine = "rasterio")
# rastest3 = np.fromfile("/moonbow/ascheb/les/SNODAS/zz_ssmv01025SlL01T0024TTNATS2010010105DP001.dat")

In [ ]:
jan1snow = xr.open_dataset("/moonbow/ascheb/les/SNODAS/01jansnowprecip.nc") #Band1 is liquid equivalent of snowfall, in mm
jan2snow = xr.open_dataset("/moonbow/ascheb/les/SNODAS/02jansnowprecip.nc")
jan3snow = xr.open_dataset("/moonbow/ascheb/les/SNODAS/03jansnowprecip.nc")

In [ ]:
afiletest = xr.open_dataset("/moonbow/ascheb/les/2010/hires_control/processed_data/mergedvars_2010-01-02-060000.nc")

In [ ]:
srftempvals = afiletest["SrfTemp"].values
srftempds = xr.Dataset(coords = afiletest["SrfTemp"].coords)
srftempds = srftempds.rename({"lat2d": "lat", "lon2d": "lon"})
srftempds = srftempds.drop(["lon1d", "lat1d"])
srftempds["SrfTemp"] = xr.DataArray(srftempvals, coords = srftempds.coords, dims = ["y", "x"])
# srftempds = srftempds.assign_coords({"lat": (("y", "x"), afiletest["lat2d"].values), "lon": (("y", "x"), afiletest["lon2d"].values)})

In [ ]:
afiletest.close(); del afiletest

In [ ]:
from time import perf_counter
# st = perf_counter()
# rgrid = xesmf.Regridder(jan1snow, srftempds, "bilinear")
# gridweights = rgrid.to_netcdf("snodasgridweights.nc")
# del gridweights
# et = perf_counter()
# print(f"regridding took {et-st:.2f} seconds!")
st = perf_counter()
gridweights = xr.open_dataset("snodasgridweights.nc")
rgrid2 = xesmf.Regridder(jan1snow, srftempds, "bilinear", weights = gridweights)
jan1snowregrid = rgrid2(jan1snow["Band1"]); jan1snowregrid.name = "Snowfall"
jan2snowregrid = rgrid2(jan2snow["Band1"]); jan2snowregrid.name = "Snowfall"
jan3snowregrid = rgrid2(jan3snow["Band1"]); jan3snowregrid.name = "Snowfall"

print(jan1snowregrid)
jan1snowregrid.to_netcdf("/moonbow/ascheb/les/SNODAS/01jansnowprecip_regrid.nc")
jan2snowregrid.to_netcdf("/moonbow/ascheb/les/SNODAS/02jansnowprecip_regrid.nc")
jan3snowregrid.to_netcdf("/moonbow/ascheb/les/SNODAS/03jansnowprecip_regrid.nc")
et = perf_counter()
print(f"regridding took {et-st:.2f} seconds!")

In [ ]:
del jan1snowregrid; del jan2snowregrid; del jan3snowregrid

In [ ]:
from palettable.lightbartlein.sequential import Blues10_10
nohrsccmap = Blues10_10.get_mpl_colormap()
snowbounds = [1e-3, 0.1, 0.5, 1, 2, 3, 4, 5]
snownorm = BoundaryNorm(snowbounds, ncolors = 256, extend = "max")
plt.pcolormesh(jan3snowregrid/100, norm = snownorm, cmap = nohrsccmap)

In [ ]:
jan1snowregrid = xr.open_dataset("/moonbow/ascheb/les/SNODAS/01jansnowprecip_regrid.nc")
jan2snowregrid = xr.open_dataset("/moonbow/ascheb/les/SNODAS/02jansnowprecip_regrid.nc")
jan3snowregrid = xr.open_dataset("/moonbow/ascheb/les/SNODAS/03jansnowprecip_regrid.nc")


In [ ]:
print(jan1snow.lat)
print(np.argwhere((jan1snow.lat.values>=afile_nolake["lat1d"].min().values)*(jan1snow.lat.values<=afile_nolake["lat1d"].max().values)))
print(np.argwhere((jan1snow.lon.values>=afile_nolake["lon1d"].min().values)*(jan1snow.lon.values<=afile_nolake["lon1d"].max().values)))

In [ ]:
print(jan3snow["Band1"].sel(lat = 43.00, lon = -81.8, method = "nearest")/254)

In [ ]:
plt.pcolormesh((jan3snow["Band1"][1556:2740, 5339:6785]/254).where(jan3snow["Band1"][1556:2740, 5339:6785]/254<0.04).values)

In [ ]:
# from palettable.lightbartlein.sequential import Blues10_10
# nohrsccmap = Blues10_10.get_mpl_colormap()
# snowbounds = [0.04, 0.2, 0.39, 0.79, 1.2, 1.6, 2.0, 3.9, 5.9, 7.9, 9.8, 14, 20]
# snownorm = BoundaryNorm(snowbounds, ncolors = 256)
# plt.figure(figsize = (10, 12))
# plt.contour(afile_control["lon1d"], afile_control["lat1d"], afile_control["Patch"][1,:,:], levels = [0.999], colors = "black", linewidths = 0.5, zorder = 2)
# plt.pcolormesh(jan2snow["lon"][5339:6785], jan2snow["lat"][1556:2740], (jan3snow["Band1"][1556:2740, 5339:6785]/254).where(jan3snow["Band1"][1556:2740, 5339:6785]/254>0.04), cmap = nohrsccmap, norm = snownorm, zorder = 1)
# plt.pcolormesh(jan2snow["lon"][5339:6785], jan2snow["lat"][1556:2740], np.ones((1183, 1445)), color = "lightgrey", zorder = 0)

In [ ]:
print(et-st)

In [ ]:
ftime = "03-0500"
# afile_nolake = xr.open_dataset(f"/moonbow/ascheb/les/2010/hires_nolake/processed_data/mergedvars_2010-01-{ftime}00.nc")
afile_control = xr.open_dataset(f"/moonbow/ascheb/les/2010/hires_control/processed_data/mergedvars_2010-01-{ftime}00.nc")
# afile_nolake_old = xr.open_dataset(f"/moonbow/ascheb/les/2010/hires_nolake/processed_data/mergedvars_2010-01-02-060000.nc")
afile_control_old = xr.open_dataset(f"/moonbow/ascheb/les/2010/hires_control/processed_data/mergedvars_2010-01-02-060000.nc")
snowdiff_control = afile_control["SnowPrecipTotal"]+afile_control["AggPrecipTotal"]-afile_control_old["SnowPrecipTotal"]-afile_control_old["AggPrecipTotal"]
# snowdiff_nolake = afile_nolake["SnowPrecipTotal"]+afile_nolake["AggPrecipTotal"]-afile_nolake_old["SnowPrecipTotal"]-afile_nolake_old["AggPrecipTotal"]

In [ ]:
from palettable.cmocean.sequential import Ice_10
snowcmap = Ice_10.get_mpl_colormap()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 14))
fakecontour = Line2D([], [], color = "black", linewidth = 1, label = "Water Bodies")
for ax in fig.get_axes():
    ax.contour(afile_control["lon1d"], afile_control["lat1d"], afile_control["Patch"][1,:,:], levels = [0.999], colors = "black", linewidths = 1, zorder = 2)
    # ax.pcolormesh(jan2snow["lon"][5339:6785], jan2snow["lat"][1556:2740], np.ones((1183, 1445)), color = "black", zorder = 0)
    ax.pcolormesh(afile_control["lon1d"], afile_control["lat1d"], afile_control["Topo"], norm = MidpointNormalize(vmin = -20, vmax = 1000, vcenter = 0), cmap = "gist_earth", zorder = 0)
    ax.set_xlabel("Longitude (Degrees E)")
    ax.set_ylabel("Latitude (Degrees N)")
    ax.set_yticks([41, 42, 43, 44, 45, 46])
    ax.set_xticks([-84, -82, -80])
    ax.legend(loc = "lower left", fontsize = 20, handles = [fakecontour])
from palettable.lightbartlein.sequential import Blues10_10
nohrsccmap = Blues10_10.get_mpl_colormap()
snowbounds = [1e-3, 0.1, 0.5, 1, 2, 3, 4, 5]
snownorm = BoundaryNorm(snowbounds, ncolors = 256, extend = "max")
snowmp = ax1.pcolormesh(afile_control["lon1d"], afile_control["lat1d"], (jan3snowregrid["Snowfall"]/100).where((jan3snowregrid["Snowfall"]/100)>1e-3), cmap = nohrsccmap, norm = snownorm, zorder = 1)
#factor of 100 converts to *cm* of liquid equivalent. Raw data is in kg/m^2/10, so need to multiply by 10 to get mm. I'm using cm because that's the NOHRSc scale
# ax1.pcolormesh(jan2snow["lon"][5339:6785], jan2snow["lat"][1556:2740], np.ones((1183, 1445)), color = "lightgrey", zorder = 0)
# ax1.pcolormesh(afile_nolake["lon1d"], afile_nolake["lat1d"], jan1snowregrid, cmap = snowcmap, vmin = 0, vmax = 30)
ax2.pcolormesh(afile_control["lon1d"], afile_control["lat1d"], (snowdiff_control/10).where(snowdiff_control/10>1e-3), cmap = nohrsccmap, norm = snownorm, zorder = 1)
# ax3.pcolormesh(afile_nolake["lon1d"], afile_nolake["lat1d"], (snowdiff_nolake/10).where(snowdiff_nolake/10>0.1), cmap = nohrsccmap, norm = snownorm, zorder = 1)
ax1.set_title("SNODAS 24-Hour Snowfall ")
ax2.set_title("Simulated 24-Hour Snowfall")
fig.suptitle("24-Hour Snowfall from 2010-01-02 0500 UTC to 2010-01-03 0500 UTC")
cbar = fig.colorbar(snowmp, ax = [ax1, ax2], orientation = "horizontal", fraction = 0.07, label = "24-Hour Snowfall (cm Liquid Equivalent)"); cbar.ax.set_xticklabels([0, 0.1, 0.5, 1, 2, 3, 4, 5])
fig.savefig("PaperFigs/snowverif.png")

In [ ]:
print(jan3snowregrid["__xarray_dataarray_variable__"])